In [2]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model

# Define image dimensions
img_width, img_height = 100, 100

# Loading and preprocessing images for training
X_train = []
y_train = []

for folder, label in [('nished_images', 0), ('susan_images', 1)]:
    for filename in os.listdir(folder):
        img = load_img(os.path.join(folder, filename), target_size=(img_width, img_height))
        img_array = img_to_array(img)
        X_train.append(img_array)
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification (self or friend)
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluating the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

# Saving the model
model.save('my_face_recognition_model.keras')

# Loading the pre-trained face recognition model
model = load_model('my_face_recognition_model.keras')

# Loading the pre-trained deep learning-based face detection model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000.caffemodel")

def preprocess_image(img):
    """
    Preprocesses the input image for model prediction.

    Parameters:
    - img: Input image.

    Returns:
    - img: Preprocessed image.
    """
    img = cv2.resize(img, (100, 100))  # Resizing to match model input size
    img = img.astype('float32') / 255.0  # Normalizing pixel values
    return img

def recognize_faces(img):
    """
    Recognizes faces in the input image and draws rectangles and labels.

    Parameters:
    - img: Input image.

    Returns:
    - img: Image with rectangles and labels drawn around recognized faces.
    """
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:  # Adjusting confidence threshold as needed
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face_roi = img[startY:endY, startX:endX]  # Region of Interest (ROI) for the face
            face_roi_processed = preprocess_image(face_roi)
            prediction = model.predict(np.expand_dims(face_roi_processed, axis=0))
            
            # Assuming your model output is binary (1 for your face, 0 for others)
            if prediction[0][0] > 0.5:
                cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)  # Green rectangle for your face
                cv2.putText(img, 'Nished', (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.rectangle(img, (startX, startY), (endX, endY), (0, 0, 255), 2)  # Red rectangle for Susan's face
                cv2.putText(img, 'Susan', (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    return img

# Capturing video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = recognize_faces(frame)
    cv2.imshow('Face Recognition', frame)

    # Checking if the user pressed 'q' to quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Releasing the webcam and closing all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Epoch 1/10
8/8 [==============================] - 7s 618ms/step - loss: 37.7694 - accuracy: 0.5458 - val_loss: 1.3147 - val_accuracy: 0.7705
Epoch 2/10
8/8 [==============================] - 4s 521ms/step - loss: 0.4659 - accuracy: 0.8875 - val_loss: 0.1340 - val_accuracy: 0.9508
Epoch 3/10
8/8 [==============================] - 5s 626ms/step - loss: 0.1800 - accuracy: 0.9417 - val_loss: 0.0765 - val_accuracy: 0.9836
Epoch 4/10
8/8 [==============================] - 5s 614ms/step - loss: 0.0250 - accuracy: 0.9958 - val_loss: 0.0439 - val_accuracy: 0.9836
Epoch 5/10
8/8 [==============================] - 5s 568ms/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0531 - val_accuracy: 0.9672
Epoch 6/10
8/8 [==============================] - 5s 654ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0824 - val_accuracy: 0.9672
Epoch 7/10
8/8 [==============================] - 7s 820ms/step - loss: 9.6005e-04 - accuracy: 1.0000 - val_loss: 0.1810 - val_accuracy: 0.9508
Epoch 8/10
8/8 

1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 31ms/step
